# Load Spacy Model

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")
import json

In [64]:
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import pickle
from pathlib import Path
import warnings 
warnings.filterwarnings('ignore')
import numpy as np

In [8]:
res_file = open("Entity Recognition in Resumes.json",'r',encoding='utf-8')
json_str = []
for line in res_file:
    json_str.append(line)

In [27]:
json.loads(json_str[1])

{'content': 'Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• 

# Prepare Data For Spacy

In [ ]:
# This function to remove overlaping entities
def filter_anotation(anLst):
    ss_n = []
    prev_ij = []
    
    for i,j,s in anLst:
        if prev_ij == []:
            prev_ij.append([i,j])
            ss_n.append((i,j,s))
        else:
            not_pre = True
            for ii in prev_ij:
                a = ii[0]
                b = ii[1]
                if ((i >= a and i <b) or (j >= a and j <b)) or ((a >= i and a <j) or (b >= i and b <j)):
                    not_pre = False
                    
            if [i,j] not in prev_ij and not_pre:
                prev_ij.append([i,j])
                ss_n.append((i,j,s))
    return ss_n


In [39]:
sk_set = []
lr = []
OL = []
labels = []
for json_entr in json_str:
    d = json.loads(json_entr)
    cont = " ".join(d['content'].split("\n"))
    cont  = " ".join([sd.text.lower() for sd in nlp.tokenizer(cont) if not sd.is_punct])
    ets = []
    for ano in d['annotation']:
        if len(ano['label']) != 0:            
            lbl = "_".join(ano['label'][0].lower().split(" "))
            
            txt =  " ".join(ano['points'][0]['text'].split('\n'))
            txt = [sd.text for sd in nlp.tokenizer(" ".join(txt.split(","))) if not sd.is_punct]
            txt = [t.text.lower() for t in nlp.tokenizer(ano['points'][0]['text']) if t.is_alpha]

            for ww in txt:
                s = cont.find(ww)
                e = s + len(ww)
                if s!=-1:
                    OL.append([ww,lbl])
                    if e-s >2:
                        tpl = (s,e,lbl)
                        ets.append(tpl)
    ets = filter_anotation(ets)
    ets = list(set(ets))
    if len(ets) != 0:
        lr.append((cont,{'entities':ets}))
        

In [65]:
n = len(lr)
test_per = 0.2
train_data = lr[:int(n*(1-test_per))]
test_data = lr[-1*int(n*test_per):]

In [66]:
len(train_data),len(test_data)

(176, 44)

In [122]:
test_data[5]

('nida khan tech support executive teleperformance for microsoft   jaipur rajasthan email me on indeed indeed.com/r/nida-khan/6c9160696f57efd8   to be an integral part of the organization and enhance my knowledge to utilize it in a productive manner for the growth of the company and the global   industrial training   bhel heep haridwar on cnc system&amp plc programming   work experience   tech support executive   teleperformance for microsoft   september 2017 to present   process 21 months of experience in adfc as phone banker   education   bachelor of technology in electronics communication engg   gnit institute of technology   lucknow uttar pradesh   2008 to 2012   class xii   u.p. board   bareilly uttar pradesh   2007   class x   u.p. board   bareilly uttar pradesh   2005   skills   microsoft office excel cisco c language cbs 4 years   https://www.indeed.com/r/nida-khan/6c9160696f57efd8?isid=rex-download&ikw=download-top&co=in',
 {'entities': [(557, 567, 'college_name'),
   (805, 81


# Prepare new NER Spacy Pipeline

In [73]:
test_nlp = spacy.blank('en')
ner = test_nlp.create_pipe('ner')
test_nlp.add_pipe('ner')

In [74]:
for _,et in train_data:
    for sss in et['entities']:
        lbl = sss[-1]
        ner.add_label(lbl)

In [75]:
optimizer = test_nlp.begin_training()

In [76]:
p= 0
batch_size = 10
for _ in range(400):
    losses = {}
    random.shuffle(train_data)
    
    for i in range(0,len(train_data),batch_size):
        batch = []
        for raw_text, entity_offsets in train_data[i:i+batch_size]:
            doc = test_nlp.make_doc(raw_text)
            example = Example.from_dict(doc, entity_offsets)
            batch.append(example)
        test_nlp.update(batch, sgd=optimizer, drop=0.2,
                   losses=losses)
    p = p+1
    print('Losses', losses,p)

Losses {'ner': 46402.300721591106} 1
Losses {'ner': 10557.318391839042} 2
Losses {'ner': 9932.518139360473} 3
Losses {'ner': 9492.379184351303} 4
Losses {'ner': 9227.639436833793} 5
Losses {'ner': 8585.594196146354} 6
Losses {'ner': 8364.281489412882} 7
Losses {'ner': 8071.230324698612} 8
Losses {'ner': 7713.988535086624} 9
Losses {'ner': 7382.056466132868} 10
Losses {'ner': 7188.287369651953} 11
Losses {'ner': 7670.2755635991925} 12
Losses {'ner': 6822.523830083024} 13
Losses {'ner': 6913.090414646082} 14
Losses {'ner': 6649.963975582694} 15
Losses {'ner': 6357.061282712617} 16
Losses {'ner': 6123.230789102847} 17
Losses {'ner': 5899.534422313212} 18
Losses {'ner': 5749.906565129553} 19
Losses {'ner': 5615.244502707152} 20
Losses {'ner': 5486.020955686021} 21
Losses {'ner': 5547.594596983616} 22
Losses {'ner': 5420.022360139759} 23
Losses {'ner': 5277.461785773281} 24
Losses {'ner': 5059.464160461928} 25
Losses {'ner': 4989.017748712096} 26
Losses {'ner': 4951.846625936669} 27
Losses 

Losses {'ner': 804.7281479980868} 218
Losses {'ner': 725.6503929908797} 219
Losses {'ner': 746.5868445308811} 220
Losses {'ner': 678.6879661030772} 221
Losses {'ner': 719.409944490706} 222
Losses {'ner': 642.734251948451} 223
Losses {'ner': 744.6949780019154} 224
Losses {'ner': 709.9120589267825} 225
Losses {'ner': 699.0537855876871} 226
Losses {'ner': 714.8432278457489} 227
Losses {'ner': 698.3932113088287} 228
Losses {'ner': 671.0344483882427} 229
Losses {'ner': 680.4467061715659} 230
Losses {'ner': 660.3281443250664} 231
Losses {'ner': 651.5837940127758} 232
Losses {'ner': 613.2484442930512} 233
Losses {'ner': 712.2924743968268} 234
Losses {'ner': 669.2309881658089} 235
Losses {'ner': 674.7401082176979} 236
Losses {'ner': 715.9757620440179} 237
Losses {'ner': 637.3145963191317} 238
Losses {'ner': 677.1958763114419} 239
Losses {'ner': 662.605094666964} 240
Losses {'ner': 687.5041447597362} 241
Losses {'ner': 676.5560402363747} 242
Losses {'ner': 678.9234597920187} 243
Losses {'ner': 

# Test Accuracy

In [107]:
train_all_accuracy = []
for resume_data in train_data:
    content = resume_data[0]
    anot = resume_data[1]['entities']
    words = [content[an[0]:an[1]] for an in anot]
    nlp_content = test_nlp(content)
    
    found_ents = [e.text for e in nlp_content.ents]
    correct_found = [e for e in found_ents if e in words]
    eta = len(correct_found)/len(words)
    train_all_accuracy.append(eta)
acc = np.mean(train_all_accuracy)
print("Train Accuracy:",acc*100)

Train Accuracy: 97.52532454504941


In [106]:
dd = test_nlp(train_data[10][0])
spacy.displacy.render(dd,style='ent')

In [108]:
all_accuracy = []
for resume_data in test_data:
    content = resume_data[0]
    anot = resume_data[1]['entities']
    words = [content[an[0]:an[1]] for an in anot]
    nlp_content = test_nlp(content)
    
    found_ents = [e.text for e in nlp_content.ents]
    correct_found = [e for e in found_ents if e in words]
    eta = len(correct_found)/len(words)
    all_accuracy.append(eta)
acc = np.mean(all_accuracy)
print("Test Accuracy:",acc*100)

Test Accuracy: 53.367343091299546


In [115]:
dd = test_nlp(test_data[38][0])
spacy.displacy.render(dd,style='ent')

In [119]:
spacy.__version__

'3.1.4'